In [2]:
from pyspark.sql import SparkSession
ss = SparkSession.builder.appName("SparkAttaques").master("spark://spark-master:7077")\
    .config("spark.executor.memory","2g")\
    .config("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.0")\
    .config("spark.mongodb.output.uri","mongodb://mongo:27017/joueur_saison.attaques")\
    .getOrCreate()
sc = ss.sparkContext

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b08bea60-d07c-4e42-b72e-b583161e7f86;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.0 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 234ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.0 from central in [default]
	-

In [18]:
hdfs_url = "hdfs://namenode/"
stat_gene = ss.read.csv(hdfs_url + "user/root/StatistiquesGene.csv",header = True, sep = ",")

In [17]:
decisives = stat_gene.select("joueur","Annees","Goals","Decisives Passes","Non Penalty Goals","Penalty Goals","GCA")\
            .toPandas()\
            .fillna(0)\
            .set_index(["joueur","Annees"])\
            .astype(int)\
            .sum(axis = 1)

decisives = decisives.reset_index()
decisives.columns = ["Joueur","Annees","Decisives"]

In [16]:
non_decisives = stat_gene.select("joueur","Annees","Penalty Attempteds","Shoots Total","Shoots On Target","SCA")\
                .toPandas()\
                .fillna(0)\
                .set_index(["joueur","Annees"])\
                .astype(int)\
                .sum(axis = 1)

non_decisives = non_decisives.reset_index()
non_decisives.columns = ["Joueur","Annees","Non Decisives"]

In [6]:
attaque = decisives.merge(non_decisives, on=['Joueur','Annees'])

In [20]:
pip install PyMongo
import pandas as pd
import pymongo
from pymongo import MongoClient

In [25]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://mongo:27017/joueur_saison.attaques")
# database
db = client["joueur_saison"]
# collection
attaques= db["attaques"]
# Insertion in Mongo

db.attaques.insert_many(attaque.to_dict('record'))

TypeError: documents must be a non-empty list